In [1]:
# import keras
# from keras import layers
# from keras import ops
# from keras.layers import TextVectorization
import numpy as np
import pandas as pd
import os
import string
import random
import tensorflow as tf
import tensorflow.data as tf_data
import tensorflow.strings as tf_strings
import matplotlib.pyplot as plt

os.environ["KERAS_BACKEND"] = "tensorflow"

2024-03-14 12:22:35.408087: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-14 12:22:35.412943: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 12:22:35.569371: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-03-14 12:22:36.252093: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-14 12:22:38.354908: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF

In [2]:
df = pd.read_pickle('../midi_df_2199.pkl')

In [3]:
MAX_TOKENS = 128
BUFFER_SIZE = df.shape[0]
BATCH_SIZE = 64

In [4]:
# pad_to_max_len_list = max(df['tokenized_guitar'].apply(lambda x : len(x)))

In [ ]:
plt.hist(df['tokenized_guitar'].apply(lambda x : len(x)).sort_values(ascending=False), bins=100)

In [6]:
df.shape

(2199, 7)

In [7]:
# df['tokenized_guitar'] = df['tokenized_guitar'].apply(lambda x: np.asarray(x).astype(np.float32))
# df['tokenized_drums'] = df['tokenized_drums'].apply(lambda x: np.asarray(x).astype(np.float32))

In [8]:
# df['tokenized_guitar'] = df['tokenized_guitar'].apply(lambda x: np.pad(x, pad_width=(0, (pad_to_max_len_list-len(x))), mode='constant', constant_values=0))
# df['tokenized_drums'] = df['tokenized_drums'].apply(lambda x: np.pad(x, pad_width=(0, (pad_to_max_len_list-len(x))), mode='constant', constant_values=0))

In [9]:
# len(df['tokenized_drums'][190])

In [99]:
train_ratio = 0.8

train_index = int(round((df.shape[0]*train_ratio)//BATCH_SIZE, 0)*BATCH_SIZE)
test_size = int((((df.shape[0])-train_index)//BATCH_SIZE)*BATCH_SIZE)
test_index = train_index + test_size

In [ ]:
train_data = df.iloc[:train_index].loc[:,['tokenized_guitar', 'tokenized_drums']]
test_data = df.iloc[train_index:test_index].loc[:,['tokenized_guitar', 'tokenized_drums']]

In [ ]:
# train_data['tokenized_guitar'] = train_data['tokenized_guitar'].apply(lambda x: tf.constant((x)))
# train_data['tokenized_drums'] = train_data['tokenized_drums'].apply(lambda x: tf.constant((x)))

In [ ]:
train_data['tokenized_guitar'][0]

In [ ]:
# ds_train = tf.data.Dataset.from_tensors((tf.ragged.constant(train_data['tokenized_guitar']).to_tensor(),
#                                                           tf.ragged.constant(train_data['tokenized_drums']).to_tensor()))

# ds_test = tf.data.Dataset.from_tensors((tf.ragged.constant(test_data['tokenized_guitar']).to_tensor(),
#                                                           tf.ragged.constant(test_data['tokenized_drums']).to_tensor()))

In [ ]:
lst=[]

for guitar,drums in zip(tf.split(tf.ragged.constant(test_data['tokenized_guitar']).to_tensor(),test_size,axis=0),tf.split(tf.ragged.constant(test_data['tokenized_drums']).to_tensor(),test_size,axis=0)):
    lst.append((guitar,drums))

ds_test = tf.data.experimental.from_list(lst).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
lst=[]

for guitar,drums in zip(tf.split(tf.ragged.constant(train_data['tokenized_guitar']).to_tensor(),train_index,axis=0),tf.split(tf.ragged.constant(train_data['tokenized_drums']).to_tensor(),train_index,axis=0)):
    lst.append((guitar,drums))
    
ds_train = tf.data.experimental.from_list(lst).prefetch(buffer_size=tf.data.AUTOTUNE)

In [ ]:
def prepare_batch(guitar, drums):
    guitar = guitar[:, :, :MAX_TOKENS]    # Trim to MAX_TOKENS.

    drums = drums[:, :, :(MAX_TOKENS+1)]
    drums_input = drums[:, :, :-1]
    drums_labels = drums[:, :, 1:]

    return (tf.squeeze(guitar), tf.squeeze(drums_input)), tf.squeeze(drums_labels)

In [ ]:
def make_batches(ds):
  return (
      ds
      .shuffle(BUFFER_SIZE)
      .batch(BATCH_SIZE)
      .map(prepare_batch, tf.data.AUTOTUNE)
      .map(lambda *x: tf.nest.map_structure(lambda c: tf.ensure_shape(c, [BATCH_SIZE, MAX_TOKENS]), x))
      .prefetch(buffer_size=tf.data.AUTOTUNE))

In [ ]:
# Create training and validation set batches.
train_batches = make_batches(ds_train)
val_batches = make_batches(ds_test)

In [ ]:
train_batches.take(1).get_single_element()

In [ ]:
def positional_encoding(length, depth):
  depth = depth/2

  positions = np.arange(length)[:, np.newaxis]     # (seq, 1)
  depths = np.arange(depth)[np.newaxis, :]/depth   # (1, depth)
  
  angle_rates = 1 / (10000**depths)         # (1, depth)
  angle_rads = positions * angle_rates      # (pos, depth)

  pos_encoding = np.concatenate(
      [np.sin(angle_rads), np.cos(angle_rads)],
      axis=-1) 

  return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
#@title
pos_encoding = positional_encoding(length=2048, depth=512)

# Check the shape.
print(pos_encoding.shape)

# Plot the dimensions.
plt.pcolormesh(pos_encoding.numpy().T, cmap='RdBu')
plt.ylabel('Depth')
plt.xlabel('Position')
plt.colorbar()
plt.show()

In [ ]:
#@title
pos_encoding/=tf.norm(pos_encoding, axis=1, keepdims=True)
p = pos_encoding[1000]
dots = tf.einsum('pd,d -> p', pos_encoding, p)
plt.subplot(2,1,1)
plt.plot(dots)
plt.ylim([0,1])
plt.plot([950, 950, float('nan'), 1050, 1050],
         [0,1,float('nan'),0,1], color='k', label='Zoom')
plt.legend()
plt.subplot(2,1,2)
plt.plot(dots)
plt.xlim([950, 1050])
plt.ylim([0,1])

In [ ]:
class PositionalEmbedding(tf.keras.layers.Layer):
  def __init__(self, vocab_size, d_model):
    super().__init__()
    self.d_model = d_model
    self.embedding = tf.keras.layers.Embedding(vocab_size, d_model, mask_zero=True) 
    self.pos_encoding = positional_encoding(length=2048, depth=d_model)

  def compute_mask(self, *args, **kwargs):
    return self.embedding.compute_mask(*args, **kwargs)

  def call(self, x):
    length = tf.shape(x)[1]
    x = self.embedding(x)
    # This factor sets the relative scale of the embedding and positonal_encoding.
    x *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))
    x = x + self.pos_encoding[tf.newaxis, :length, :]
    return x

In [ ]:
guitar_vocab_size = 169499
drums_vocab_size = 213357

In [ ]:
embed_guitar = PositionalEmbedding(vocab_size=guitar_vocab_size, d_model=512)
embed_drums = PositionalEmbedding(vocab_size=drums_vocab_size, d_model=512)

guitar_emb = embed_guitar(guitar)
drums_emb = embed_drums(drums)

In [ ]:
drums_emb._keras_mask

In [ ]:
class BaseAttention(tf.keras.layers.Layer):
  def __init__(self, **kwargs):
    super().__init__()
    self.mha = tf.keras.layers.MultiHeadAttention(**kwargs)
    self.layernorm = tf.keras.layers.LayerNormalization()
    self.add = tf.keras.layers.Add()

In [ ]:
class CrossAttention(BaseAttention):
  def call(self, x, context):
    attn_output, attn_scores = self.mha(
        query=x,
        key=context,
        value=context,
        return_attention_scores=True)
   
    # Cache the attention scores for plotting later.
    self.last_attn_scores = attn_scores

    x = self.add([x, attn_output])
    x = self.layernorm(x)

    return x

In [ ]:
sample_ca = CrossAttention(num_heads=2, key_dim=512)

print(guitar_emb.shape)
print(drums_emb.shape)
print(sample_ca(drums_emb, guitar_emb).shape)

In [ ]:
class GlobalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
sample_gsa = GlobalSelfAttention(num_heads=2, key_dim=512)

print(guitar_emb.shape)
print(sample_gsa(guitar_emb).shape)

In [ ]:
class CausalSelfAttention(BaseAttention):
  def call(self, x):
    attn_output = self.mha(
        query=x,
        value=x,
        key=x,
        use_causal_mask = True)
    x = self.add([x, attn_output])
    x = self.layernorm(x)
    return x

In [ ]:
sample_csa = CausalSelfAttention(num_heads=2, key_dim=512)

print(drums_emb.shape)
print(sample_csa(drums_emb).shape)

In [ ]:
# out1 = sample_csa(drums_emb(drums[:, :3])) 
# out2 = sample_csa(drums_emb(drums))[:, :3]

# tf.reduce_max(abs(out1 - out2)).numpy()

In [ ]:
class FeedForward(tf.keras.layers.Layer):
  def __init__(self, d_model, dff, dropout_rate=0.1):
    super().__init__()
    self.seq = tf.keras.Sequential([
      tf.keras.layers.Dense(dff, activation='relu'),
      tf.keras.layers.Dense(d_model),
      tf.keras.layers.Dropout(dropout_rate)
    ])
    self.add = tf.keras.layers.Add()
    self.layer_norm = tf.keras.layers.LayerNormalization()

  def call(self, x):
    x = self.add([x, self.seq(x)])
    x = self.layer_norm(x) 
    return x

In [ ]:
sample_ffn = FeedForward(512, 2048)

print(drums_emb.shape)
print(sample_ffn(drums_emb).shape)

In [ ]:
class EncoderLayer(tf.keras.layers.Layer):
  def __init__(self,*, d_model, num_heads, dff, dropout_rate=0.1):
    super().__init__()

    self.self_attention = GlobalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x):
    x = self.self_attention(x)
    x = self.ffn(x)
    return x

In [ ]:
sample_encoder_layer = EncoderLayer(d_model=512, num_heads=8, dff=2048)

print(guitar_emb.shape)
print(sample_encoder_layer(guitar_emb).shape)

In [ ]:
class Encoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads,
               dff, vocab_size, dropout_rate=0.1):
    super().__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(
        vocab_size=vocab_size, d_model=d_model)

    self.enc_layers = [
        EncoderLayer(d_model=d_model,
                     num_heads=num_heads,
                     dff=dff,
                     dropout_rate=dropout_rate)
        for _ in range(num_layers)]
    self.dropout = tf.keras.layers.Dropout(dropout_rate)

  def call(self, x):
    # `x` is token-IDs shape: (batch, seq_len)
    x = self.pos_embedding(x)  # Shape `(batch_size, seq_len, d_model)`.
    
    # Add dropout.
    x = self.dropout(x)

    for i in range(self.num_layers):
      x = self.enc_layers[i](x)

    return x  # Shape `(batch_size, seq_len, d_model)`.

In [ ]:
# Instantiate the encoder.
sample_encoder = Encoder(num_layers=4,
                         d_model=512,
                         num_heads=8,
                         dff=2048,
                         vocab_size=guitar_vocab_size)

sample_encoder_output = sample_encoder(guitar, training=False)

# Print the shape.
print(guitar.shape)
print(sample_encoder_output.shape)  # Shape `(batch_size, input_seq_len, d_model)`.

In [ ]:
class DecoderLayer(tf.keras.layers.Layer):
  def __init__(self,
               *,
               d_model,
               num_heads,
               dff,
               dropout_rate=0.1):
    super(DecoderLayer, self).__init__()

    self.causal_self_attention = CausalSelfAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)
    
    self.cross_attention = CrossAttention(
        num_heads=num_heads,
        key_dim=d_model,
        dropout=dropout_rate)

    self.ffn = FeedForward(d_model, dff)

  def call(self, x, context):
    x = self.causal_self_attention(x=x)
    x = self.cross_attention(x=x, context=context)

    # Cache the last attention scores for plotting later
    self.last_attn_scores = self.cross_attention.last_attn_scores

    x = self.ffn(x)  # Shape `(batch_size, seq_len, d_model)`.
    return x

In [ ]:
sample_decoder_layer = DecoderLayer(d_model=512, num_heads=8, dff=2048)

sample_decoder_layer_output = sample_decoder_layer(
    x=drums_emb, context=guitar_emb)

print(drums_emb.shape)
print(guitar_emb.shape)
print(sample_decoder_layer_output.shape)  # `(batch_size, seq_len, d_model)`

In [ ]:
class Decoder(tf.keras.layers.Layer):
  def __init__(self, *, num_layers, d_model, num_heads, dff, vocab_size,
               dropout_rate=0.1):
    super(Decoder, self).__init__()

    self.d_model = d_model
    self.num_layers = num_layers

    self.pos_embedding = PositionalEmbedding(vocab_size=vocab_size,
                                             d_model=d_model)
    self.dropout = tf.keras.layers.Dropout(dropout_rate)
    self.dec_layers = [
        DecoderLayer(d_model=d_model, num_heads=num_heads,
                     dff=dff, dropout_rate=dropout_rate)
        for _ in range(num_layers)]

    self.last_attn_scores = None

  def call(self, x, context):
    # `x` is token-IDs shape (batch, target_seq_len)
    x = self.pos_embedding(x)  # (batch_size, target_seq_len, d_model)

    x = self.dropout(x)

    for i in range(self.num_layers):
      x  = self.dec_layers[i](x, context)

    self.last_attn_scores = self.dec_layers[-1].last_attn_scores

    # The shape of x is (batch_size, target_seq_len, d_model).
    return x

In [ ]:
# Instantiate the decoder.
sample_decoder = Decoder(num_layers=4,
                         d_model=512,
                         num_heads=8,
                         dff=2048,
                         vocab_size=drums_vocab_size)

output = sample_decoder(
    x=drums,
    context=guitar_emb)

# Print the shapes.
print(drums.shape)
print(guitar_emb.shape)
print(output.shape)

In [ ]:
sample_decoder.last_attn_scores.shape  # (batch, heads, target_seq, input_seq)

In [ ]:
class Transformer(tf.keras.Model):
  def __init__(self, *, num_layers, d_model, num_heads, dff,
               input_vocab_size, target_vocab_size, dropout_rate=0.1):
    super().__init__()
    self.encoder = Encoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=input_vocab_size,
                           dropout_rate=dropout_rate)

    self.decoder = Decoder(num_layers=num_layers, d_model=d_model,
                           num_heads=num_heads, dff=dff,
                           vocab_size=target_vocab_size,
                           dropout_rate=dropout_rate)

    self.final_layer = tf.keras.layers.Dense(target_vocab_size)

  def call(self, inputs):
    # To use a Keras model with `.fit` you must pass all your inputs in the
    # first argument.
    context, x  = inputs

    context = self.encoder(context)  # (batch_size, context_len, d_model)

    x = self.decoder(x, context)  # (batch_size, target_len, d_model)

    # Final linear layer output.
    logits = self.final_layer(x)  # (batch_size, target_len, target_vocab_size)

    try:
      # Drop the keras mask, so it doesn't scale the losses/metrics.
      # b/250038731
      del logits._keras_mask
    except AttributeError:
      pass

    # Return the final output and the attention weights.
    return logits

In [ ]:
num_layers = 4
d_model = 128
dff = 512
num_heads = 8
dropout_rate = 0.1

In [ ]:
transformer = Transformer(
    num_layers=num_layers,
    d_model=d_model,
    num_heads=num_heads,
    dff=dff,
    input_vocab_size=guitar_vocab_size,
    target_vocab_size=drums_vocab_size,
    dropout_rate=dropout_rate)

In [ ]:
output = transformer((guitar, drums))

print(drums.shape)
print(guitar.shape)
print(output.shape)

In [ ]:
attn_scores = transformer.decoder.dec_layers[-1].last_attn_scores
print(attn_scores.shape)  # (batch, heads, target_seq, input_seq)

In [ ]:
transformer.summary()

In [ ]:
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
  def __init__(self, d_model, warmup_steps=4000):
    super().__init__()

    self.d_model = d_model
    self.d_model = tf.cast(self.d_model, tf.float32)

    self.warmup_steps = warmup_steps

  def __call__(self, step):
    step = tf.cast(step, dtype=tf.float32)
    arg1 = tf.math.rsqrt(step)
    arg2 = step * (self.warmup_steps ** -1.5)

    return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
learning_rate = CustomSchedule(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate, beta_1=0.9, beta_2=0.98,
                                     epsilon=1e-9)

In [ ]:
plt.plot(learning_rate(tf.range(40000, dtype=tf.float32)))
plt.ylabel('Learning Rate')
plt.xlabel('Train Step')

In [ ]:
def masked_loss(label, pred):
  mask = label != 0
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
  loss = loss_object(label, pred)

  mask = tf.cast(mask, dtype=loss.dtype)
  loss *= mask

  loss = tf.reduce_sum(loss)/tf.reduce_sum(mask)
  return loss


def masked_accuracy(label, pred):
  pred = tf.argmax(pred, axis=2)
  label = tf.cast(label, pred.dtype)
  match = label == pred

  mask = label != 0

  match = match & mask

  match = tf.cast(match, dtype=tf.float32)
  mask = tf.cast(mask, dtype=tf.float32)
  return tf.reduce_sum(match)/tf.reduce_sum(mask)

In [ ]:
transformer.compile(
    loss=masked_loss,
    optimizer=optimizer,
    metrics=[masked_accuracy])

In [ ]:
transformer.fit(train_batches,
                epochs=20,
                validation_data=val_batches)

In [ ]:
tf.saved_model.save(transformer, 'taxifare_pablopussell/model/')

In [2]:
transformer = tf.saved_model.load('model/')

In [96]:
class Translator(tf.Module):
  def __init__(self, transformer):
    self.transformer = transformer

  def __call__(self, guitar_tokens, max_length=MAX_TOKENS):
    # The input is a tokenized guitar track.
    guitar_tensor = tf.ragged.constant(guitar_tokens)
    assert isinstance(guitar_tensor, tf.Tensor)
    if len(guitar_tensor.shape) == 0:
        guitar_tensor = guitar_tensor[tf.newaxis]

    print(guitar_tensor)
    encoder_input = guitar_tensor

    # As the output is a tokenized drum track, initialize the output with the
    # drum's `[START]` token.
    start_token = tf.ragged.constant(1)
    end_token = tf.ragged.constant(6)
    start = start_token[tf.newaxis]
    end = end_token[tf.newaxis]
      
    # `tf.TensorArray` is required here (instead of a Python list), so that the
    # dynamic-loop can be traced by `tf.function`.
    output_array = tf.TensorArray(dtype=tf.int32, size=0, dynamic_size=True)
    
    output_array = output_array.write(0, start)
    print(output_array)

    for i in tf.range(max_length):
      output = tf.transpose(output_array.stack())
      predictions = self.transformer((encoder_input, output), training=False)

      # Select the last token from the `seq_len` dimension.
      predictions = predictions[:, -1:, :]  # Shape `(batch_size, 1, vocab_size)`.

      predicted_id = tf.argmax(predictions, axis=-1)

      # Concatenate the `predicted_id` to the output which is given to the
      # decoder as its input.
      output_array = output_array.write(i+1, predicted_id[0])

      if predicted_id == end:
        break

    output = tf.transpose(output_array.stack())
    # The output shape is `(1, tokens)`.
    drum_tokenized = output[1]  # Shape: `()`.

    # `tf.function` prevents us from using the attention_weights that were
    # calculated on the last iteration of the loop.
    # So, recalculate them outside the loop.
    self.transformer([encoder_input, output[:,:-1]], training=False)
    attention_weights = self.transformer.decoder.last_attn_scores

    return tokens, attention_weights

In [97]:
translator = Translator(transformer)

In [3]:
transformer.save_weights('model/model_weights/')

AttributeError: '_UserObject' object has no attribute 'save_weights'

In [81]:
df.iloc[2]['tokenized_guitar'][0] = 1
df.iloc[2]['tokenized_guitar'][-1] = 6


In [82]:
# pred_ds = tf.data.experimental.from_list([df.iloc[2]['tokenized_guitar']])

In [83]:
# pred_ds.take(1).get_single_element()

In [98]:
drum_tokens, attention_weights = translator(df.iloc[2]['tokenized_guitar'])

tf.Tensor(
[     1      5      5      5      5      5      5      5      5      5
      5      5      5      5      5 144952 143487 142160  80835 139724
 138545 137405  78116 135241 134245 133173  75680 131321 130401 129565
  73134 127847 127081 126419 125782 125169 124543 123968 123387 122920
 122318  66933 121300 120821 120354  64953  64409 118985 118511 118092
 117697 117337 116976  60888  60359 115933 115638 115350 115066 114779
 114452 114228 113991      5      5      5      5      5      5      5
      5 112345 112163  52760  52535 111674 111546 111427  51133 111193
 111063  50140  49777 110753 110639 110554 110470 110372 110275 110207
 110069 110028 109958 109899  46372 109783 109655  45546 109550 109504
 109438  44489 109332 109295 109248  43557  43279 109098 109035 108990
  42323 108895 108844  41607 108746      6], shape=(116,), dtype=int32)


ValueError: Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (1 total):
    * (<tf.Tensor 'inputs:0' shape=(116,) dtype=int32>,
 <tf.Tensor 'inputs_1:0' shape=(1, 1) dtype=int32>)
  Keyword arguments: {'training': False}

 Expected these arguments to match one of the following 4 option(s):

Option 1:
  Positional arguments (1 total):
    * (TensorSpec(shape=(None, 921), dtype=tf.int32, name='input_1'),
 TensorSpec(shape=(None, 921), dtype=tf.int32, name='input_2'))
  Keyword arguments: {'training': False}

Option 2:
  Positional arguments (1 total):
    * (TensorSpec(shape=(None, 921), dtype=tf.int32, name='inputs_0'),
 TensorSpec(shape=(None, 921), dtype=tf.int32, name='inputs_1'))
  Keyword arguments: {'training': False}

Option 3:
  Positional arguments (1 total):
    * (TensorSpec(shape=(None, 921), dtype=tf.int32, name='inputs_0'),
 TensorSpec(shape=(None, 921), dtype=tf.int32, name='inputs_1'))
  Keyword arguments: {'traini